In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import pyNastran
from pyNastran.op2.op2 import OP2
from pyNastran.bdf.bdf import BDF

In [ ]:
def ReadOP2Files(path):
    path = path + '*.op2'
    print('Given File path is : \n' , path , '\n')
    op2ListName = glob.glob(path)
    print('Found the following list of OP2 Files: \n')
    print('========================================', '\n')
    op2ListName
    dictOP2FileList = {}
    for i in range(len(op2ListName)):
        #print(colors[i])
        dictOP2FileList[os.path.splitext(os.path.basename(op2ListName[i]))[0]] = op2ListName[i]
        #print(os.path.basename(op2ListName[i]), op2ListName[i])
        #print(os.path.splitext(os.path.basename(op2ListName[i])))
    return dictOP2FileList

In [ ]:
def ReadOP2File(op2FileName):
    preOP2Model = OP2()
    preOP2File = op2FileName
    # Load Pre OP2 File
    preOP2Model.read_op2(preOP2File, build_dataframe=True)
    return preOP2Model

In [ ]:
def ReadBDFFile(bdfFileName):
    fileName = os.path.splitext(bdfFileName)[0] + '.bdf'
    model = BDF()
    model.read_bdf(fileName,xref=True)
    return model

In [ ]:
def ReadLoadcases(preOP2Model):
    print(preOP2Model.get_op2_stats(short=True))
    preLCs = list(preOP2Model.cshear_stress.keys())
    return preLCs

In [ ]:
# Function Which creates new index for Shear Stress individual loadcase
def newShearIndex(lc):
    colReInx = pd.MultiIndex.from_tuples([(lc, 'max_shear'), (lc, 'avg_shear'), (lc, 'margin')])
    return colReInx

In [ ]:
def ReadShearStressResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)
    # Start with an empty dataframe [note: Logic to be changed later]
    cShearResultDF_Pre = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cshear_stress[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newShearIndex(lc), level=1)
        if cShearResultDF_Pre.empty == False:
            cShearResultDF_Pre = pd.concat([cShearResultDF_Pre, tempDF1], axis=1,sort=False)
        else :
            cShearResultDF_Pre = tempDF1.copy()
    return cShearResultDF_Pre

In [ ]:
# Function Which creates new index for Bar Stress individual loadcase
def newBarIndex(lc):
    colReInx = pd.MultiIndex.from_tuples([(lc, 's1a'), (lc, 's2a'), (lc, 's3a'), (lc, 's4a'), (lc, 'axial'), (lc, 'smaxa'), (lc, 'smina'), (lc, 'MS_tension'), (lc, 's1b'), (lc, 's2b'), (lc, 's3b'), (lc, 's4b'), (lc, 'smaxb'), (lc, 'sminb'), (lc, 'MS_compression')])
    return colReInx

In [ ]:
def ReadBarStressResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)
    # Start with an empty dataframe [note: Logic to be changed later]
    cBarResultDF_Pre = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cbar_stress[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newBarIndex(lc), level=1)
        if cBarResultDF_Pre.empty == False:
            cBarResultDF_Pre = pd.concat([cBarResultDF_Pre, tempDF1], axis=1,sort=False)
        else :
            cBarResultDF_Pre = tempDF1.copy()
    return cBarResultDF_Pre
    

In [ ]:
# Function Which creates new index for Bar Stress individual loadcase
def newBarIndex_Axial(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'axial')])
    return colReInx

In [ ]:
def ReadBarAxialStressResuls(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)
    # Start with an empty dataframe [note: Logic to be changed later]
    cBarResultDF_Pre_Axial = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cbar_stress[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newBarIndex_Axial(lc), level=1)
        if cBarResultDF_Pre_Axial.empty == False:
            cBarResultDF_Pre_Axial = pd.concat([cBarResultDF_Pre_Axial, tempDF1], axis=1,sort=False)
        else :
            cBarResultDF_Pre_Axial = tempDF1.copy()    
    return cBarResultDF_Pre_Axial

In [ ]:
# Function Which creates new index for Quad Stress individual loadcase
def newQUADIndex_Fx(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'oxx')])
    return colReInx
# Function Which creates new index for Quad Stress individual loadcase
def newQUADIndex_Fy(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'oyy')])
    return colReInx
# Function Which creates new index for Quad Stress individual loadcase
def newQUADIndex_Fxy(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'txy')])
    return colReInx

In [ ]:
def ReadShellFxResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)    
    # Start with an empty dataframe [note: Logic to be changed later]
    cShellResultDF_Pre_Fx = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cquad4_stress[lc].data_frame
        tempDF1 = tempDF.reindex(columns=newQUADIndex_Fx(lc), level=1)
        if cShellResultDF_Pre_Fx.empty == False:
            cShellResultDF_Pre_Fx = pd.concat([cShellResultDF_Pre_Fx, tempDF1], axis=1,sort=False)
        else :
            cShellResultDF_Pre_Fx = tempDF1.copy()    
    return cShellResultDF_Pre_Fx

In [ ]:
def ReadShellFyResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)     
    # Start with an empty dataframe [note: Logic to be changed later]
    cShellResultDF_Pre_Fy = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cquad4_stress[lc].data_frame
        tempDF1 = tempDF.reindex(columns=newQUADIndex_Fy(lc), level=1)
        if cShellResultDF_Pre_Fy.empty == False:
            cShellResultDF_Pre_Fy = pd.concat([cShellResultDF_Pre_Fy, tempDF1], axis=1,sort=False)
        else :
            cShellResultDF_Pre_Fy = tempDF1.copy()
    return cShellResultDF_Pre_Fy    

In [ ]:
def ReadShellFxyResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)    
    # Start with an empty dataframe [note: Logic to be changed later]
    cShellResultDF_Pre_Fxy = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.cquad4_stress[lc].data_frame
        tempDF1 = tempDF.reindex(columns=newQUADIndex_Fxy(lc), level=1)
        if cShellResultDF_Pre_Fxy.empty == False:
            cShellResultDF_Pre_Fxy = pd.concat([cShellResultDF_Pre_Fxy, tempDF1], axis=1,sort=False)
        else :
            cShellResultDF_Pre_Fxy = tempDF1.copy()
    return cShellResultDF_Pre_Fxy

In [ ]:
# Function Which creates new index for Rod Axial Stress individual loadcase
def newRodIndex_Axial(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'axial')])
    return colReInx

In [ ]:
def ReadRodAxialStressResults(preOP2Model):
    preLCs = ReadLoadcases(preOP2Model)     
    # Start with an empty dataframe [note: Logic to be changed later]
    cRodResultDF_Pre_Axial = pd.DataFrame({'A' : []})
    for lc in preLCs:
        tempDF = preOP2Model.crod_stress[lc].data_frame
        tempDF = tempDF.unstack(level=0).T
        tempDF1 = tempDF.reindex(columns=newRodIndex_Axial(lc), level=1)
        if cRodResultDF_Pre_Axial.empty == False:
            cRodResultDF_Pre_Axial = pd.concat([cRodResultDF_Pre_Axial, tempDF1], axis=1,sort=False)
        else :
            cRodResultDF_Pre_Axial = tempDF1.copy()    
    return cRodResultDF_Pre_Axial

In [ ]:
def AppendRodStressToBarStressDataFrame(cBarResultDF_Pre_Axial,cRodResultDF_Pre_Axial):
    Appended_Pre_Results = cBarResultDF_Pre_Axial.append(cRodResultDF_Pre_Axial, ignore_index=False)
    return Appended_Pre_Results

In [ ]:
# ===========================================
# Main Start of OP2 Files Extraction == v0.0
# Remarks := Its been depricated.
# Author := Upendra Reddy G
# ===========================================

# dictOp2List = ReadOP2Files('./')
# for key, op2File in dictOp2List.items():
#     #print(key, op2File)
#     op2Model = ReadOP2File(op2File)

#     barStressFileName = key + '_cBar_Stress.csv'
#     barAxialStressFileName = key + '_cBar_Stress_Axial.csv'
#     quadStressFxFileName = key + '_cQUAD_Stress_Fx.csv'
#     quadStressFyFileName = key + '_cQUAD_Stress_Fy.csv'
#     quadStressFxyFileName = key + '_cQUAD_Stress_Fxy.csv'
#     rodAxialStressFileName = key + '_cRod_Stress_Axial.csv'
#     appendedBarRodstressFileName = key + '_Appended_cBar_and_Rod_Stresses_Axial.csv'
    
    
#     barStresResults = ReadBarStressResults(op2Model)
#     barStresResults.to_csv(barStressFileName)

#     barAxialStresResults = ReadBarAxialStressResuls(op2Model)
#     barAxialStresResults.to_csv(barAxialStressFileName)
    
#     quadFxStresResults = ReadShellFxResults(op2Model)
#     quadFxStresResults.to_csv(quadStressFxFileName)
    
#     quadFyStresResults = ReadShellFyResults(op2Model)
#     quadFyStresResults.to_csv(quadStressFyFileName)    
        
#     quadFxyStresResults = ReadShellFxyResults(op2Model)
#     quadFxyStresResults.to_csv(quadStressFxyFileName)
    
#     rodAxialStressResults = ReadRodAxialStressResults(op2Model)
#     rodAxialStressResults.to_csv(rodAxialStressFileName)
    
#     appendRodToBarResults = AppendRodStressToBarStressDataFrame(barAxialStresResults,rodAxialStressResults)
#     appendRodToBarResults.to_csv(appendedBarRodstressFileName)

# print(' ======== ALL DONE SUCCESSFULLY ========== ')

In [ ]:
def Concatenate_Data(df1,df2):
    if df1.empty == False:
        df1 = pd.concat([df1, df2], axis=1,sort=False)
    else :
        df1 = df2.copy()
    return df1

In [ ]:
def Join_DF(df1,df2):
    if df1.empty == True:
        df1 = df2.copy()
    else :
        df1 = df1.join(df2, how='outer')
    return df1

In [ ]:
def Join_DF(df1,df2):
    if df1.empty == True:
        df1 = df2.copy()
    else :
        df1 = df1.join(df2, how='outer')
    return df1

In [ ]:
# # ===========================================
# # Main Start of OP2 Files Extraction == v1.0
# # Remarks := Currently Working..
# # Author := Upendra Reddy G
# # ===========================================
# barStressResults = pd.DataFrame({'A' : []})
# barAxialStressResults = pd.DataFrame({'A' : []})
# quadFxStressResults = pd.DataFrame({'A' : []})
# quadFyStressResults = pd.DataFrame({'A' : []})
# quadFxyStressResults = pd.DataFrame({'A' : []})
# rodAxialStressResults = pd.DataFrame({'A' : []})
# shearStressResults = pd.DataFrame({'A' : []})
# appendRodToBarResults = pd.DataFrame({'A' : []})

# dictOp2List = ReadOP2Files('./')

# # The Following loop will fetch results from every Available OP2 File
# for key, op2File in dictOp2List.items():
#     #print(key, op2File)
#     op2Model = ReadOP2File(op2File)

#     tempDF1 = ReadBarStressResults(op2Model)
#     barStressResults = Join_DF(barStressResults, tempDF1)

#     tempDF1 = ReadBarAxialStressResuls(op2Model)
#     barAxialStressResults = Join_DF(barAxialStressResults, tempDF1)

#     tempDF1 = ReadShellFxResults(op2Model)
#     quadFxStressResults = Join_DF(quadFxStressResults, tempDF1)
    
#     tempDF1 = ReadShellFyResults(op2Model)
#     quadFyStressResults = Join_DF(quadFyStressResults, tempDF1)
    
#     tempDF1 = ReadShellFxyResults(op2Model)
#     quadFxyStressResults = Join_DF(quadFxyStressResults, tempDF1)
    
#     tempDF1 = ReadRodAxialStressResults(op2Model)
#     rodAxialStressResults = Join_DF(rodAxialStressResults, tempDF1)
    
#     tempDF1 = ReadShearStressResults(op2Model)
#     shearStressResults = Join_DF(shearStressResults, tempDF1)    
    

# #  The Following line of code will append Rod Axial Stress results to Bar Axial Stress Results    
# appendRodToBarResults = barAxialStressResults.append(rodAxialStressResults, ignore_index=False)

# # The Following lines of code will generate csv files for respective results.
# barStressResults.to_csv('ALL_cBar_Stress.csv')
# barAxialStressResults.to_csv('ALL_cBar_Stress_Axial.csv')
# quadFxStressResults.to_csv('ALL_cQUAD_Stress_Fx.csv')
# quadFyStressResults.to_csv('ALL_cQUAD_Stress_Fy.csv')
# quadFxyStressResults.to_csv('ALL_cQUAD_Stress_Fxy.csv')
# rodAxialStressResults.to_csv('ALL_cRod_Stress_Axial.csv')
# shearStressResults.to_csv('ALL_cShear_Stress.csv')
# appendRodToBarResults.to_csv('ALL_appended_Rod_to_Bar_Stress_Axial.csv')

# # End of Script.
# print(' ======== ALL DONE SUCCESSFULLY ========== ')

# The following code will extract calculated <font color='red'>beam stresses</font> using input bdf & output op2 files.

In [ ]:
# Function Which creates new index for calculated beam Axial Stress
def beam_new_index(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'Location'), (lc, 'sd'), (lc, 'axial_force'), (lc, 'element_type'), (lc, 'property_id'), (lc, 'element_area'), (lc, 'beam_stress')])
    return colReInx

In [ ]:
def Calculate_Beam_Stresses(modeOP2, modelbdf):
    preLCs = ReadLoadcases(modeOP2)
    cBeamResultDF_Axial = pd.DataFrame({'A' : []})
    for lc in preLCs:
        beam_stresses_df = modeOP2.cbeam_force[lc].data_frame
#     modeOP2.cbeam_force.keys()
        beam_stresses_df['element_type'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].type)
        beam_stresses_df['property_id'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].pid)
        beam_stresses_df['element_area'] = beam_stresses_df.property_id.map(lambda x: modelbdf.properties[x].Area())
        beam_stresses_df['beam_stress'] = beam_stresses_df.axial_force / beam_stresses_df.element_area
        interested_columns  =['ElementID', 'Location', 'sd', 'axial_force', 'element_type', 'property_id', 'element_area', 'beam_stress']
        beam_stresses_df = beam_stresses_df[interested_columns]
        beam_stresses_df.ElementID = beam_stresses_df.ElementID.astype(np.int64)
        beam_stresses_df.set_index('ElementID', inplace=True)
        beam_stresses_df = beam_stresses_df.reindex(columns=beam_new_index(lc), level=1)
        
        if cBeamResultDF_Axial.empty == False:
            cBeamResultDF_Axial = pd.concat([cBeamResultDF_Axial, beam_stresses_df], axis=1,sort=False)
        else :
            cBeamResultDF_Axial = beam_stresses_df.copy()
#     beam_stresses_df.head()
    return cBeamResultDF_Axial

# The following code will extract calculated <font color='red'>beam Axial stresses</font> using input bdf & output op2 files.

In [ ]:
# Function Which creates new index for calculated beam Axial Stress
def beam_new_index_for_Axial(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'beam_stress')])
    return colReInx

In [ ]:
def Calculate_Beam_Stresses_Axial_Only(modeOP2, modelbdf):
    preLCs = ReadLoadcases(modeOP2)
    cBeamResultDF_Axial = pd.DataFrame({'A' : []})
    for lc in preLCs:
        beam_stresses_df = modeOP2.cbeam_force[lc].data_frame
# #     modeOP2.cbeam_force.keys()
#         beam_stresses_df['element_type'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].type)
#         beam_stresses_df['property_id'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].pid)
#         beam_stresses_df['element_area'] = beam_stresses_df.property_id.map(lambda x: modelbdf.properties[x].Area())
#         beam_stresses_df['beam_stress'] = beam_stresses_df.axial_force / beam_stresses_df.element_area
#         interested_columns  =['ElementID', 'beam_stress']
#         beam_stresses_df = beam_stresses_df[interested_columns]
#         beam_stresses_df.ElementID = beam_stresses_df.ElementID.astype(np.int64)
#         beam_stresses_df.set_index('ElementID', inplace=True)
#         beam_stresses_df = beam_stresses_df.reindex(columns=beam_new_index_for_Axial(lc), level=1)
# #         interested_columns  =[ 'beam_stress']
#         tempDF = beam_stresses_df.beam     
        beam_stresses_df = beam_stresses_df.reset_index()
        beam_stresses_df['element_type'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].type)
        beam_stresses_df['property_id'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].pid)
        beam_stresses_df['element_area'] = beam_stresses_df.property_id.map(lambda x: modelbdf.properties[x].Area())
        beam_stresses_df['beam_stress'] = beam_stresses_df.axial_force / beam_stresses_df.element_area
        interested_columns  =['ElementID', 'beam_stress']
        beam_stresses_df = beam_stresses_df[interested_columns]
        beam_stresses_df.ElementID = beam_stresses_df.ElementID.astype(np.int64)
        beam_stresses_df.set_index('ElementID', inplace=True)
        beam_stresses_df = beam_stresses_df.reindex(columns=beam_new_index_for_Axial(lc), level=1)        
        
        if cBeamResultDF_Axial.empty == False:
            cBeamResultDF_Axial = pd.concat([cBeamResultDF_Axial, beam_stresses_df], axis=1,sort=False)
        else :
            cBeamResultDF_Axial = beam_stresses_df.copy()
#     beam_stresses_df.head()
    return cBeamResultDF_Axial

# The following code will create <font color='red'> Combined Bar and Beam Axial stresses </font> using input bdf & output op2 files.

In [ ]:
# Function Which creates new index for calculated beam Axial Stress
def beam_new_index_for_Axial_append(lc):
    colReInx = pd.MultiIndex.from_tuples([ (lc, 'axial')])
    return colReInx

In [ ]:
def Beam_Stresses_Axial_For_append(modeOP2, modelbdf):
    preLCs = ReadLoadcases(modeOP2)
    cBeamResultDF_Axial = pd.DataFrame({'A' : []})
    for lc in preLCs:
        beam_stresses_df = modeOP2.cbeam_force[lc].data_frame
        #         beam_stresses_df = beam_stresses_df.reset_index()
        beam_stresses_df['element_type'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].type)
        beam_stresses_df['property_id'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].pid)
        beam_stresses_df['element_area'] = beam_stresses_df.property_id.map(lambda x: modelbdf.properties[x].Area())
        beam_stresses_df['axial'] = beam_stresses_df.axial_force / beam_stresses_df.element_area
        interested_columns = ['ElementID', 'axial']
        beam_stresses_df = beam_stresses_df[interested_columns]
        beam_stresses_df.ElementID = beam_stresses_df.ElementID.astype(np.int64)
        beam_stresses_df.set_index('ElementID', inplace=True)
        beam_stresses_df = beam_stresses_df.reindex(columns=beam_new_index_for_Axial_append(lc), level=1)
        # beam_stresses_df.head()  
        
        if cBeamResultDF_Axial.empty == False:
            cBeamResultDF_Axial = pd.concat([cBeamResultDF_Axial, beam_stresses_df], axis=1,sort=False)
        else :
            cBeamResultDF_Axial = beam_stresses_df.copy()
#     beam_stresses_df.head()
    return cBeamResultDF_Axial

# <font color='green'> Main Start </font>
## <font color='blue'> ============ </font>

In [ ]:
# ===========================================
# Main Start of Beam Stresses by Calculation [P/A]
# Remarks := Currently Working..
# Author := Upendra Reddy G
# Date := 23/01/2019
# ===========================================

beamStressResults_Calculated = pd.DataFrame({'A' : []})
beamStressResults_Axial_Only_Calculated = pd.DataFrame({'A' : []})
beamStressResults_Axial_Only_Calculated_append = pd.DataFrame({'A' : []})
barAxialStressResults = pd.DataFrame({'A' : []})

dictOp2List = ReadOP2Files('./')

# The Following loop will fetch results from every Available OP2 File
for key, op2File in dictOp2List.items():
    #print(key, op2File)
    op2Model = ReadOP2File(op2File)
    bdfModel = ReadBDFFile(op2File)
    
    tempDF1 = ReadBarAxialStressResuls(op2Model)
    barAxialStressResults = Join_DF(barAxialStressResults, tempDF1)    
    
    tempDF1 = Calculate_Beam_Stresses(op2Model, bdfModel)
    beamStressResults_Calculated = Concatenate_Data(beamStressResults_Calculated, tempDF1)
    
    tempDF1 = Calculate_Beam_Stresses_Axial_Only(op2Model, bdfModel)
    beamStressResults_Axial_Only_Calculated = Concatenate_Data(beamStressResults_Axial_Only_Calculated, tempDF1)    
    
    tempDF1 = Beam_Stresses_Axial_For_append(op2Model, bdfModel)
    beamStressResults_Axial_Only_Calculated_append = Concatenate_Data(beamStressResults_Axial_Only_Calculated_append, tempDF1)    

beamStressResults_Calculated.to_csv('ALL_Beam_Stress_Results_Calculated.csv')
beamStressResults_Axial_Only_Calculated.to_csv('ALL_Beam_Stress_Results_Axial_Only_Calculated.csv')

beamCalcs = beamStressResults_Axial_Only_Calculated_append.copy()
beamCalcs = beamCalcs.reset_index()
beamCalcs['Static'] = 0
beamCalcs = beamCalcs.set_index(['Static', 'ElementID'])

Bar_Beam_Combined_Results = pd.concat([barAxialStressResults, beamCalcs])
Bar_Beam_Combined_Results.to_csv('ALL_Bar_Beam_Combined_Axial_Stress_Results.csv')


# End of Script.
print(' ======== ALL DONE SUCCESSFULLY ========== ')

# <font color='green'> Test Work </font>
## <font color='blue'> ============ </font>

In [ ]:
temp = beamStressResults_Axial_Only_Calculated_append.copy()
temp2 = temp.reset_index()
temp2['Static'] = 0
temp2 = temp2.set_index(['Static', 'ElementID'])
temp2.head()


# temp = beamStressResults_Axial_Only_Calculated_append.copy()
# temp3 = temp.reset_index()
# temp3['Static'] = 0
# temp3 = temp3.set_index(['Static', 'ElementID'])
# # temp3.head()

# comb = pd.concat([temp3, temp2])
# # comb.head()



In [ ]:
barAxialStressResults.head()

In [ ]:
temp = barAxialStressResults.copy()
# temp3 = temp.reset_index()
# temp3['Static'] = 0
# temp3 = temp3.set_index(['Static', 'ElementID'])
# temp3.head()

In [ ]:
comb = pd.concat([temp, temp2])
comb.head()

In [ ]:
comb.shape

In [ ]:
op2File = 'pc12-47-v1-6-static_post_m3.2_2101.op2'
lc = 92101

modeOP2 = ReadOP2File(op2File)
modelbdf = ReadBDFFile(op2File)

tempDF1 = ReadBarAxialStressResuls(op2Model)
barAxialStressResults = Join_DF(barAxialStressResults, tempDF1)

beam_stresses_df = modeOP2.cbeam_force[lc].data_frame
#          beam_stresses_df = beam_stresses_df.reset_index()
beam_stresses_df['element_type'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].type)
beam_stresses_df['property_id'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].pid)
beam_stresses_df['element_area'] = beam_stresses_df.property_id.map(lambda x: modelbdf.properties[x].Area())
beam_stresses_df['axial'] = beam_stresses_df.axial_force / beam_stresses_df.element_area
interested_columns = ['ElementID', 'axial']
beam_stresses_df = beam_stresses_df[interested_columns]
beam_stresses_df.ElementID = beam_stresses_df.ElementID.astype(np.int64)
beam_stresses_df.set_index('ElementID', inplace=True)
beam_stresses_df = beam_stresses_df.reindex(columns=beam_new_index_for_Axial_append(lc), level=1)
beam_stresses_df.head()

In [ ]:
beam_stresses_df  = beam_stresses_df.reset_index()
beam_stresses_df['Static'] = 0
# beamStressResults_Axial_Only_Calculated_append = beamStressResults_Axial_Only_Calculated_append.set_index(['Static', 'ElementID'])
barAxialStressResults = barAxialStressResults.reset_index()
# beamStressResults_Axial_Only_Calculated_append  = beamStressResults_Axial_Only_Calculated_append.reset_index()
tempDF = pd.concat([barAxialStressResults, beamStressResults_Axial_Only_Calculated_append])
tempDF = tempDF.set_index(['Static', 'ElementID'])
tempDF.head()

In [ ]:
barAxialStressResults.head()

In [ ]:
op2File = 'pc12-47-v1-6-static_post_m3.2_2101.op2'
lc = 92101

modeOP2 = ReadOP2File(op2File)
modelbdf = ReadBDFFile(op2File)

beam_stresses_df = modeOP2.cbeam_force[lc].data_frame
#     modeOP2.cbeam_force.keys()
beam_stresses_df['element_type'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].type)
beam_stresses_df['property_id'] = beam_stresses_df.ElementID.map(lambda x: modelbdf.elements[x].pid)
beam_stresses_df['element_area'] = beam_stresses_df.property_id.map(lambda x: modelbdf.properties[x].Area())
beam_stresses_df['beam_stress'] = beam_stresses_df.axial_force / beam_stresses_df.element_area
interested_columns  =['ElementID', 'Location', 'sd', 'axial_force', 'element_type', 'property_id', 'element_area', 'beam_stress']
beam_stresses_df = beam_stresses_df[interested_columns]
beam_stresses_df.ElementID = beam_stresses_df.ElementID.astype(np.int64)
beam_stresses_df.set_index('ElementID', inplace=True)
beam_stresses_df = beam_stresses_df.reindex(columns=beam_new_index_for_Axial_append(lc), level=1)

In [ ]:
beam_stresses_df.head()